In [1]:
%load_ext autoreload
%autoreload 2

In [48]:
import numpy as np
import json
from equistore import Labels, TensorBlock, TensorMap
from utils.builder import DescriptorBuilder
from utils.acdc import cg_combine
import ase.io
from itertools import product
from utils.clebsh_gordan import ClebschGordanReal
from utils.hamiltonians import fix_pyscf_l1, dense_to_blocks, blocks_to_dense, couple_blocks, decouple_blocks
import matplotlib.pyplot as plt
from utils.librascal import  RascalSphericalExpansion, RascalPairExpansion

In [3]:
frames = ase.io.read("data/water-hamiltonian/water_coords_1000.xyz",":")
for f in frames:
    f.cell = [100,100,100]
    f.positions += 50

In [4]:
jorbs = json.load(open('data/water-hamiltonian/orbs_def2_water.json', "r"))
orbs = {}
zdic = {"O" : 8, "H":1}
for k in jorbs:
    orbs[zdic[k]] = jorbs[k]

In [5]:
orbs

{8: [[1, 0, 0],
  [2, 0, 0],
  [3, 0, 0],
  [2, 1, -1],
  [2, 1, 0],
  [2, 1, 1],
  [3, 1, -1],
  [3, 1, 0],
  [3, 1, 1],
  [3, 2, -2],
  [3, 2, -1],
  [3, 2, 0],
  [3, 2, 1],
  [3, 2, 2]],
 1: [[1, 0, 0], [2, 0, 0], [2, 1, -1], [2, 1, 0], [2, 1, 1]]}

In [6]:
hams = np.load("data/water-hamiltonian/water_fock.npy", allow_pickle=True)
for i, f in enumerate(frames):
    hams[i] = fix_pyscf_l1(hams[i], f, orbs)

In [7]:
blocks = dense_to_blocks(hams, frames, orbs)

In [8]:
blocks.keys

Labels([( 0, 8, 1, 0, 8, 1, 0), ( 0, 8, 1, 0, 8, 2, 0),
        ( 0, 8, 1, 0, 8, 3, 0), ( 0, 8, 1, 0, 8, 2, 1),
        ( 0, 8, 1, 0, 8, 3, 1), ( 0, 8, 1, 0, 8, 3, 2),
        ( 0, 8, 2, 0, 8, 2, 0), ( 0, 8, 2, 0, 8, 3, 0),
        ( 0, 8, 2, 0, 8, 2, 1), ( 0, 8, 2, 0, 8, 3, 1),
        ( 0, 8, 2, 0, 8, 3, 2), ( 0, 8, 3, 0, 8, 3, 0),
        ( 0, 8, 3, 0, 8, 3, 1), ( 0, 8, 3, 0, 8, 3, 2),
        ( 0, 8, 2, 1, 8, 3, 0), ( 0, 8, 2, 1, 8, 2, 1),
        ( 0, 8, 2, 1, 8, 3, 1), ( 0, 8, 2, 1, 8, 3, 2),
        ( 0, 8, 3, 1, 8, 3, 1), ( 0, 8, 3, 1, 8, 3, 2),
        ( 0, 8, 3, 2, 8, 3, 2), ( 2, 1, 1, 0, 8, 1, 0),
        ( 2, 1, 1, 0, 8, 2, 0), ( 2, 1, 1, 0, 8, 3, 0),
        ( 2, 1, 1, 0, 8, 2, 1), ( 2, 1, 1, 0, 8, 3, 1),
        ( 2, 1, 1, 0, 8, 3, 2), ( 2, 1, 2, 0, 8, 1, 0),
        ( 2, 1, 2, 0, 8, 2, 0), ( 2, 1, 2, 0, 8, 3, 0),
        ( 2, 1, 2, 0, 8, 2, 1), ( 2, 1, 2, 0, 8, 3, 1),
        ( 2, 1, 2, 0, 8, 3, 2), ( 2, 1, 2, 1, 8, 1, 0),
        ( 2, 1, 2, 1, 8, 2, 0), ( 2, 1, 2, 1, 8,

In [9]:
blocks.block(17).values[:1]

array([[[[-0.0937688265927419],
         [-4.1469269847848725e-17],
         [0.08434071695644443],
         [-3.7793117145122225e-17],
         [0.15626116400419054]],

        [[8.477854507141878e-17],
         [-0.05567420084860641],
         [-3.637061318241603e-18],
         [-0.07947729566075579],
         [-2.0906650263738073e-17]],

        [[-0.053603092660752555],
         [-8.290087316002798e-17],
         [0.12915685084782788],
         [-6.024347747817324e-17],
         [-0.2169182693542261]]]], dtype=object)

In [10]:
cg = ClebschGordanReal(4)

In [11]:
coupled = couple_blocks(blocks, cg)

In [12]:
coupled.block(17).values.shape

(1000, 1, 1)

In [13]:
decoupled = decouple_blocks(coupled, cg)

In [14]:
decoupled.block(17).values[:1]

array([[[[-9.37688266e-02],
         [-4.14692698e-17],
         [ 8.43407170e-02],
         [-3.77931171e-17],
         [ 1.56261164e-01]],

        [[ 8.47785451e-17],
         [-5.56742008e-02],
         [-3.63706132e-18],
         [-7.94772957e-02],
         [-2.09066503e-17]],

        [[-5.36030927e-02],
         [-8.29008732e-17],
         [ 1.29156851e-01],
         [-6.02434775e-17],
         [-2.16918269e-01]]]])

In [15]:
dense = blocks_to_dense(decoupled, frames, orbs)

In [16]:
np.linalg.norm(dense[10] - hams[10])

4.944143975439669e-15

In [17]:
rascal_hypers = {
    "interaction_cutoff": 3.5,
    "cutoff_smooth_width": 0.5,
    "max_radial": 3,
    "max_angular": 2,
    "gaussian_sigma_type": "Constant",
    "compute_gradients":  False,
}

In [18]:
spex = RascalSphericalExpansion(rascal_hypers)
pairs = RascalPairExpansion(rascal_hypers)

In [19]:
rhoi = spex.compute(frames)
gij = pairs.compute(frames)

In [20]:
rhoi.keys

Labels([(0, 1, 1), (0, 1, 8), (0, 8, 1), (0, 8, 8), (1, 1, 1), (1, 1, 8),
        (1, 8, 1), (1, 8, 8), (2, 1, 1), (2, 1, 8), (2, 8, 1), (2, 8, 8)],
       dtype=[('spherical_harmonics_l', '<i4'), ('center_species', '<i4'), ('neighbor_species', '<i4')])

In [21]:
gij.block(2).samples

Labels([(  0, 0, 0), (  0, 0, 1), (  0, 0, 2), ..., (999, 2, 0),
        (999, 2, 1), (999, 2, 2)],
       dtype=[('structure', '<i4'), ('center_i', '<i4'), ('center_j', '<i4')])

In [22]:
rhoi.block(spherical_harmonics_l=1, center_species=8, neighbor_species=8).properties#values.shape

Labels([(0,), (1,), (2,)], dtype=[('n', '<i4')])

In [23]:
for idx, block in coupled:
    btype, ai, ni, li, aj, nj, lj, L = tuple(idx)
    if np.abs(btype) == 0:
        if (np.linalg.norm(block.values) < 1e-10):
            print(btype, ni, nj, li, lj, (-1)**(li+lj+L), np.linalg.norm(block.values))

In [24]:
ki, kj = 0, 0
ham_builder = DescriptorBuilder(["a1", "n1", "l1", "a2", "n2", "l2"], ["structure", "atom_i", "atom_j"], [["m1"], ["m2"]], ["hamiltonian"])
zdic = {"O": 8, "H": 1}
for A in range(len(frames)):
    frame = frames[A]
    ham = hams[A]
    ki = 0
    for i, fi in enumerate(frame.symbols):
        ai = zdic[fi]
        for ni, li, mi in jorbs[fi]:
            kj = 0
            if mi != -li:
                continue
            for j, fj in enumerate(frame.symbols):
                if i>j:
                    continue
                aj = zdic[fj]
                for nj, lj, mj in jorbs[fj]:                
                    if mj != -lj:
                        continue
                    block_idx = (ai, ni, li, aj, nj, lj)
                    if block_idx not in ham_builder.blocks:
                        block = ham_builder.add_block(sparse=block_idx,properties=np.asarray([[0]],dtype=np.int32), 
                            components=[np.asarray(list(product(range(-li,li+1)) ), dtype=np.int32 ),
                                        np.asarray(list(product(range(-lj,lj+1)) ), dtype=np.int32 )] )                                                    
                    else:
                        block = ham_builder.blocks[block_idx]
                    
                    block.add_samples(labels=[(A,i,j)], 
                                      data=np.asarray(ham[ki:ki+2*li+1, kj:kj+2*lj+1]).reshape((1,2*li+1,2*lj+1,1)))

                    kj += 2*lj+1
            ki += 2*li+1
ham_et = ham_builder.build()

In [25]:
ki, kj = 0, 0
ham_builder = DescriptorBuilder(["block_type", "a1", "n1", "l1", "a2", "n2", "l2"], ["structure", "atom_i", "atom_j", "ki_base", "kj_base"], [["m1"], ["m2"]], ["hamiltonian"])
zdic = {"O": 8, "H": 1}
for A in range(len(frames)):
    frame = frames[A]
    ham = hams[A]
    ki = 0
    for i, fi in enumerate(frame.symbols):
        ai = zdic[fi]
        ki_base = ki # pointer at where the i-atom block starts
        for ni, li, mi in jorbs[fi]:
            kj = 0
            if mi != -li:
                continue
            for j, fj in enumerate(frame.symbols):
                if i<j: # operate only on the lower-triangular block
                    continue                    
                aj = zdic[fj]
                if i==j:
                    block_type = 0  # diagonal
                elif ai==aj:
                    block_type = 1  # same-species
                else:
                    block_type = 2  # different species
                kj_base = kj # pointer at where the j-atom block starts
                for nj, lj, mj in jorbs[fj]:                
                    if mj != -lj:
                        continue
                    block_idx = (block_type, ai, ni, li, aj, nj, lj)
                    if block_idx not in ham_builder.blocks:
                        block = ham_builder.add_block(sparse=block_idx, properties=np.asarray([[0]], dtype=np.int32), 
                             components=[np.asarray(list(product(range(-li,li+1)) ), dtype=np.int32 ),
                                        np.asarray(list(product(range(-lj,lj+1)) ), dtype=np.int32 )])                                                    
                        if block_type == 1:
                            block_asym = ham_builder.add_block(sparse=(-1,)+block_idx[1:], properties=np.asarray([[0]], dtype=np.int32), 
                            components=[np.asarray(list(product(range(-li,li+1)) ), dtype=np.int32 ),
                                        np.asarray(list(product(range(-lj,lj+1)) ), dtype=np.int32 )])                                                    
                    else:                        
                        block = ham_builder.blocks[block_idx]
                        if block_type == 1:
                            block_asym = ham_builder.blocks[(-1,)+block_idx[1:]]
                    
                    block_data_ij = np.asarray(ham[ki:ki+2*li+1, kj:kj+2*lj+1])

                    if block_type == 1:
                        kj_offset = kj-kj_base
                        ki_offset = ki-ki_base
                        block_data_ji = np.asarray(ham[kj_base+ki_offset:kj_base+ki_offset+2*li+1, ki_base+kj_offset:ki_base+kj_offset+2*lj+1])                        
                        block.add_samples(labels=[(A,i,j, ki_base, kj_base)], data=(block_data_ij+block_data_ji).reshape((1,2*li+1,2*lj+1,1))/np.sqrt(2) )
                        block_asym.add_samples(labels=[(A,i,j,ki_base, kj_base)], data=(block_data_ij-block_data_ji).reshape((1,2*li+1,2*lj+1,1))/np.sqrt(2) )
                    else:
                        block.add_samples(labels=[(A,i,j,ki_base, kj_base)], data=block_data_ij.reshape((1,2*li+1,2*lj+1,1)))                    
                    kj += 2*lj+1
            ki += 2*li+1
ham_et = ham_builder.build()

In [26]:
np.linalg.norm(dense[10] - hams[10])

4.944143975439669e-15

In [27]:
dense = []
zdic = {8:"O", 1:"H"}
for f in frames:
    norbs = 0
    for ai in f.symbols:        
        norbs += len(jorbs[ai])
    ham = np.zeros((norbs, norbs), dtype=np.float64)
    dense.append(ham)

for idx, block in ham_et:
    cur_A = -1
    block_type, ai, ni, li, aj, nj, lj = tuple(idx)
    fi = zdic[ai]
    fj = zdic[aj]
    ki_offset = 0
    for no, lo, mo in jorbs[fi]:        
        if no == ni and lo == li:
            break
        ki_offset += 1
    kj_offset = 0
    for no, lo, mo in jorbs[fj]:        
        if no == nj and lo == lj:
            break
        kj_offset += 1
    for (A,i,j,ki_base,kj_base), block_data in zip(block.samples, block.values):
        if A != cur_A:
            ham = dense[A]
            cur_A = A
        if block_type == 0:
            ham[ki_base+ki_offset:ki_base+ki_offset+2*li+1, kj_base+kj_offset:kj_base+kj_offset+2*lj+1] = block_data[:,:,0].reshape(2*li+1,2*lj+1)
        elif block_type == 2:
            ham[ki_base+ki_offset:ki_base+ki_offset+2*li+1, kj_base+kj_offset:kj_base+kj_offset+2*lj+1] = block_data[:,:,0].reshape(2*li+1,2*lj+1)
            ham[kj_base+kj_offset:kj_base+kj_offset+2*lj+1, ki_base+ki_offset:ki_base+ki_offset+2*li+1] = block_data[:,:,0].reshape(2*li+1,2*lj+1).T
        elif block_type == 1:
            ham[ki_base+ki_offset:ki_base+ki_offset+2*li+1, kj_base+kj_offset:kj_base+kj_offset+2*lj+1] += np.asarray(block_data[:,:,0].reshape(2*li+1,2*lj+1)  / np.sqrt(2), dtype=np.float64)
            ham[kj_base+ki_offset:kj_base+ki_offset+2*li+1, ki_base+kj_offset:ki_base+kj_offset+2*lj+1] += np.asarray(block_data[:,:,0].reshape(2*li+1,2*lj+1) / np.sqrt(2), dtype=np.float64)
        elif block_type == -1:
            ham[ki_base+ki_offset:ki_base+ki_offset+2*li+1, kj_base+kj_offset:kj_base+kj_offset+2*lj+1] += np.asarray(block_data[:,:,0].reshape(2*li+1,2*lj+1) / np.sqrt(2), dtype=np.float64)
            ham[kj_base+ki_offset:kj_base+ki_offset+2*li+1, ki_base+kj_offset:ki_base+kj_offset+2*lj+1] -= np.asarray(block_data[:,:,0].reshape(2*li+1,2*lj+1) / np.sqrt(2), dtype=np.float64)
        



In [28]:
np.linalg.norm(dense[10] - hams[10])

2.8031757299712627e-16

In [29]:
hams[0][14]

array([1.5284446523487805, -1.3608802251214152, -1.0105189297980637,
       -0.0178662805508044, -7.95851559664695e-17, -0.6902644277689021,
       -0.043435687415332436, -8.068584708727507e-17,
       -0.40085031489053513, -0.030292005681611906, -4.31198767481943e-17,
       0.062406175805169384, -2.459992422474338e-17,
       -0.045262433044569894, -0.7229782533545261, -0.7471964897461008,
       -0.04048557118592741, -6.283581600800853e-17, 0.8773753635102233,
       -0.5021899580812228, -0.5868927205194019, 0.5208324783542648,
       -4.669393030960783e-17, -0.1388597149832118], dtype=object)

In [30]:
dense[0][14]

array([ 1.52844465e+00, -1.36088023e+00, -1.01051893e+00, -1.78662806e-02,
       -7.95851560e-17, -6.90264428e-01, -4.34356874e-02, -8.06858471e-17,
       -4.00850315e-01, -3.02920057e-02, -4.31198767e-17,  6.24061758e-02,
       -2.45999242e-17, -4.52624330e-02, -7.22978253e-01, -7.47196490e-01,
       -4.04855712e-02, -6.28358160e-17,  8.77375364e-01, -5.02189958e-01,
       -5.86892721e-01,  5.20832478e-01, -4.66939303e-17, -1.38859715e-01])

In [31]:
bb = np.where(ham_et.keys["block_type"]==-1)[0]

In [32]:
ham_et.keys[bb[0]]

(-1, 1, 1, 0, 1, 1, 0)

In [33]:
bb[0]

64

In [34]:
ham_et.block(64).values

array([[[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],


       [[[0.0]]],




In [35]:
ham_et.block(0).values.shape

(1000, 1, 1, 1)

In [36]:
ham_et.keys

Labels([( 0, 8, 1, 0, 8, 1, 0), ( 0, 8, 1, 0, 8, 2, 0),
        ( 0, 8, 1, 0, 8, 3, 0), ( 0, 8, 1, 0, 8, 2, 1),
        ( 0, 8, 1, 0, 8, 3, 1), ( 0, 8, 1, 0, 8, 3, 2),
        ( 0, 8, 2, 0, 8, 1, 0), ( 0, 8, 2, 0, 8, 2, 0),
        ( 0, 8, 2, 0, 8, 3, 0), ( 0, 8, 2, 0, 8, 2, 1),
        ( 0, 8, 2, 0, 8, 3, 1), ( 0, 8, 2, 0, 8, 3, 2),
        ( 0, 8, 3, 0, 8, 1, 0), ( 0, 8, 3, 0, 8, 2, 0),
        ( 0, 8, 3, 0, 8, 3, 0), ( 0, 8, 3, 0, 8, 2, 1),
        ( 0, 8, 3, 0, 8, 3, 1), ( 0, 8, 3, 0, 8, 3, 2),
        ( 0, 8, 2, 1, 8, 1, 0), ( 0, 8, 2, 1, 8, 2, 0),
        ( 0, 8, 2, 1, 8, 3, 0), ( 0, 8, 2, 1, 8, 2, 1),
        ( 0, 8, 2, 1, 8, 3, 1), ( 0, 8, 2, 1, 8, 3, 2),
        ( 0, 8, 3, 1, 8, 1, 0), ( 0, 8, 3, 1, 8, 2, 0),
        ( 0, 8, 3, 1, 8, 3, 0), ( 0, 8, 3, 1, 8, 2, 1),
        ( 0, 8, 3, 1, 8, 3, 1), ( 0, 8, 3, 1, 8, 3, 2),
        ( 0, 8, 3, 2, 8, 1, 0), ( 0, 8, 3, 2, 8, 2, 0),
        ( 0, 8, 3, 2, 8, 3, 0), ( 0, 8, 3, 2, 8, 2, 1),
        ( 0, 8, 3, 2, 8, 3, 1), ( 0, 8, 3, 2, 8,